In [ ]:
import sys

# !{sys.executable} -m pip install --upgrade pip
# # uncomment the following lines to install the required packages
# !{sys.executable} -m pip install seaborn
# !{sys.executable} -m pip install gmsh
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install pandas
# !{sys.executable} -m pip install Pyarrow
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install pyvista
# !{sys.executable} -m pip install ipywidgets
# !{sys.executable} -m pip install pyvirtualdisplay
# !{sys.executable} -m pip install plotly
# !{sys.executable} -m pip install -U kaleido
# !{sys.executable} -m pip install pdf2image
# !{sys.executable} -m pip install imageio
# !{sys.executable} -m pip install imageio[ffmpeg]
# !{sys.executable} -m pip install PyPDF2
# !{sys.executable} -m pip install PyMuPDF
# !{sys.executable} -m pip install IPython

# print("All packages installed!")

## Labels and colours

In [ ]:
# temperature
color_temperature = 'coolwarm'
label_temperature = r'Temperature $T$ [$^\circ$C]'

# gradient
color_gradient = 'Spectral'
label_gradient_partial = r'Gradient $\nabla T$ [$^\circ$C/m]'
label_gradient_partial_x = r'Gradient $\partial T / \partial x$ [$^\circ$C/m]'
label_gradient_partial_y = r'Gradient $\partial T / \partial y$ [$^\circ$C/m]'
label_gradient_g = r'Gradient $\mathbf g$ [$^\circ$C/m]'
label_gradient_g_x = r'Gradient $g_x$ [$^\circ$C/m]'
label_gradient_g_y = r'Gradient $g_y$ [$^\circ$C/m]'

# flux
color_flux = 'seismic'
label_flux = r'Flux $\mathbf q$ [W/m$^2$]'
label_flux_x = r'Flux $q_x$ [W/m$^2$]'
label_flux_y = r'Flux $q_y$ [W/m$^2$]'

# conductivity
color_conductivity = 'viridis'
label_conductivity = r'Thermal conductivity $k$ [W/(m$\cdot$$^\circ$C)]'

# source
color_source_min = 'black'
color_source_max = 'red'
label_source = r'Source term $s$ [W/m$^3$]'


In [ ]:
# creating custom colormap
from matplotlib.colors import LinearSegmentedColormap
# Create a custom colormap (blue to red without white)
colors = [(0, 0, 1), (1, 0, 0)]  # Blue to Red
n_bins = 100
cmap_name = 'blue_to_red'
custom_cmap_temperature = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

colors = ['green', 'yellow', 'magenta']
cmap_name = 'green_to_yellow'
custom_cmap_flux = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

colors = ['orange', 'yellow', 'purple']
cmap_name = 'orange_to_purple'
custom_cmap_grad = LinearSegmentedColormap.from_list(cmap_name, colors, N=n_bins)

custom_colorscale_temperature = [
    [0, 'blue'],
    [1, 'red']
]

custom_colorscale_flux = [
    [0, 'green'],
    [0.5, 'yellow'],
    [1, 'magenta']
]

# custom_colorscale_flux = [
#     [0, '#0000FF'],  # Blue
#     [0.5, '#FFFFFF'],  # White
#     [1, '#FF0000']  # Red
# ]

custom_colorscale_grad = [
    [0, 'orange'],
    [0.5, 'yellow'],
    [1, 'purple']
]

## Imports

In [ ]:
# maths
import numpy as np
import pandas as pd

# plotting graphs settings
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.ticker import FuncFormatter
from matplotlib.transforms import Bbox
import seaborn as sns

# sns.set_style("whitegrid")
sns.set_palette("Set2")

# plt.rcParams['figure.figsize'] = [9, 6]
plt.rcParams['figure.figsize'] = [6, 4]
plt.rcParams['figure.dpi'] = 300
# plt.rcParams['font.family'] = "Serif"
plt.rcParams['font.family'] = "Times New Roman"
plt.rcParams['font.size'] = 20

# colour pallete
# plt.rcParams['axes.prop_cycle'] = plt.cycler("color", plt.cm.tab20.colors)

SMALL_SIZE = 16
MEDIUM_SIZE = 18
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['text.usetex'] = True

# Apply the default settings to Seaborn
sns.set_context(rc=plt.rcParams)

# create mesh with gmsh
import gmsh

# plotting vtks
import pyvista as pv
pv.set_plot_theme("document")
import ipywidgets as widgets
from pyvirtualdisplay import Display

print("Imports finished! :D")

## Locations of the executables

In [ ]:

# directories
um_view = "$HOME/um_view_release" # change this to the location of your um_view_release or um_view with mofem_data_driven_finite_elements installed
tools_dir = um_view + "/bin"
dd_dir = um_view + "/mofem_data_driven_finite_elements"

# tools
read_med = tools_dir + "/read_med"
mofem_part = tools_dir + "/mofem_part"

# create datasets
create_csv_dataset = dd_dir + "/create_csv_dataset"

# classic diffusion
classic_diffusion = dd_dir + "/classic_diffusion"
hdiv_diffusion = dd_dir + "/hdiv_diffusion"

# data driven diffusion
data_driven_diffusion = dd_dir + "/data_driven_diffusion"
data_driven_diffusion_snes = dd_dir + "/data_driven_diffusion_snes"
hdiv_data_driven_diffusion = dd_dir + "/hdiv_data_driven_diffusion"
hdiv_data_driven_diffusion_snes = dd_dir + "/hdiv_data_driven_diffusion_snes"

# nonlinear diffusion
diffusion_nonlinear_graphite = dd_dir + "/diffusion_nonlinear_graphite"
diffusion_nonlinear_hdiv_graphite = dd_dir + "/diffusion_nonlinear_hdiv_graphite"
diffusion_nonlinear_snes = dd_dir + "/diffusion_nonlinear_snes"

## Params attributes 

In [ ]:
class AttrDict(dict):
    def __getattr__(self, attr):
        if attr in self:
            return self[attr]
        raise AttributeError(f"'AttrDict' object has no attribute '{attr}'")
    
params = AttrDict() # Attribute dictionary for storing the parameters

## Colobars ticks 

In [ ]:
def set_colorbar_ticks(cbar, data, decimal_places, add_min_max=True):
    # Get default ticks
    ticks = cbar.get_ticks()

    # Add min and max to default ticks if specified
    if add_min_max:
        ticks = np.unique(np.append([data.min(), data.max()], ticks))

    # Ensure ticks are within range
    ticks = ticks[(ticks >= data.min()) & (ticks <= data.max())]

    cbar.set_ticks(ticks)
    cbar.formatter = FuncFormatter(lambda x, pos: f'{x:.{decimal_places}f}'.rstrip('0').rstrip('.'))
    cbar.update_ticks()

## Paraview plots showing results

In [ ]:
def show_results(params):
    out_to_vtk = !ls -c1 {params.show_file}*vtk
    last_file=out_to_vtk[0]

    p = pv.Plotter(notebook=True)

    mesh = pv.read(last_file[:-3] + "vtk")
    warped_mesh = mesh

    if params.warp_field_scalar:
        # p.add_mesh(mesh, show_scalar_bar = False, opacity=0.2, color='gray', smooth_shading=True,)
        warped_mesh = mesh.warp_by_scalar(scalars=params.warp_field_scalar, factor=params.warp_factor)
    
    if params.warp_field_vector:
        # p.add_mesh(mesh, show_scalar_bar = False, opacity=0.2, color='gray', smooth_shading=True,)
        warped_mesh = mesh.warp_by_vector(vectors=params.warp_field_vector, factor=params.warp_factor)

    if params.show_edges:
        warped_mesh=warped_mesh.shrink(0.95)
    
    jupyter_backend='ipygany'

    # Check if the field is in point data or cell data
    if params.show_field in warped_mesh.point_data:
        data_location = warped_mesh.point_data
    elif params.show_field in warped_mesh.cell_data:
        data_location = warped_mesh.cell_data
    else:
        raise KeyError(f"Field '{params.show_field}' not found in point or cell data.")

    # TODO: Add support for showing the field part for the gradient
    # Get the scalars for coloring the points
    if (params.field_part < 0):
        scalars = data_location[params.show_field]
    elif params.field_part == 10: # plot gradient
        warped_mesh = warped_mesh.compute_derivative(scalars=params.show_field, preference='point')
        scalars = warped_mesh.point_data['gradient']
    else:
        scalars = warped_mesh.point_data[params.show_field][:,params.field_part]

    scalars = scalars * params.show_field_scale 

    if params.show_ori_shape:
        p.add_mesh(mesh, component=None, smooth_shading=True, opacity=0.5, color='gray')


    scalar_bar_title = params.show_field_name if params.show_field_name else params.show_field
    # Define the arguments for the scalar bar
    scalar_bar_args = {
        # 'n_labels': 8,
        'position_x': 0.2,  
        'title': scalar_bar_title,
    }

    p.add_mesh(warped_mesh, scalars=scalars, component=None, smooth_shading=False, cmap=params.p_cmap, clim = params.clim, show_scalar_bar=True, opacity=0.9, scalar_bar_args=scalar_bar_args)

    # if params.show_scalar_bar:
    #     scalar_bar_title = params.show_field if params.show_field else "Scalar Field"
    #     p.add_scalar_bar(title=scalar_bar_title, color='black')
 
    p.camera_position = params.camera_position
    
    p.enable_parallel_projection()
    p.enable_image_style()
    
    p.show(jupyter_backend=jupyter_backend)
    if params.p_save:
        p.save_graphic(params.p_save) 

def show_resulting_points(params):
    out_to_vtk = !ls -c1 {params.show_file}*vtk
    last_file=out_to_vtk[0]

    p = pv.Plotter(notebook=True)

    mesh = pv.read(last_file[:-3] + "vtk")

    # check if the fields are in point data or cell data or exist at all
    if params.show_field not in mesh.point_data:
        # stop function if the field does not exist
        # do not raise error just a warning and stop
        print(f"Field '{params.show_field}' not found in point data.")
        return
    
    if params.show_field_2 and params.show_field_2 not in mesh.point_data:
        # stop function if the field does not exist
        # do not raise error just a warning and stop
        print(f"Field '{params.show_field_2}' not found in point data.")
        return


    # Create a point cloud from the mesh points
    point_cloud = pv.PolyData(mesh.points)

    # Check if params.show_field_2 is defined and calculate the difference if so
    if (params.show_field_2):
        if (params.field_part < 0):
            scalars1 = mesh.point_data[params.show_field]
            scalars2 = mesh.point_data[params.show_field_2]
        else:
            scalars1 = mesh.point_data[params.show_field][:, params.field_part]
            scalars2 = mesh.point_data[params.show_field_2][:, params.field_part]
        
        scalars = scalars1 * params.show_field_scale - scalars2 * params.show_field_scale_2
        if params.show_field_diff_abs:
            scalars = np.abs(scalars)
    else:
        # Get the scalars for coloring the points
        if (params.field_part < 0):
            scalars = mesh.point_data[params.show_field]
        else:
            scalars = mesh.point_data[params.show_field][:, params.field_part]

    # Define the arguments for the scalar bar
    scalar_bar_title = params.show_field_name if params.show_field_name else params.show_field
    scalar_bar_args = {
        # 'n_labels': 8,
        'position_x': 0.2,  
        'title': scalar_bar_title,
    }

    # Add the point cloud to the plot with coloring based on the scalars
    p.add_points(point_cloud, scalars=scalars, cmap=params.p_cmap, clim=params.clim, point_size=5, scalar_bar_args=scalar_bar_args)

    p.camera_position = params.camera_position
    
    p.enable_parallel_projection()
    p.enable_image_style()
    
    p.show(jupyter_backend='ipygany')
    if params.p_save:
        p.save_graphic(params.p_save)

params.warp_field_scalar = ""    # no warp field
params.warp_field_vector = ""    # no warp field
params.warp_factor = 1.0  # warp factor
params.show_edges = True # show edges
params.triangle_mesh = True # triangle mesh
params.config_file = "bc.cfg" # config file
params.conductivity = 1.0 # linear conductivity
params.camera_position = "xy" # camera position
params.clim = None # colorbar limits
params.p_save = None
params.p_cmap = "turbo"
params.show_scalar_bar = True
params.show_field = None
params.field_part = -1
params.show_ori_shape = False
params.show_field_name = None

# for show_resulting_points with 2 fields
params.show_field_2 = None
params.show_field_scale = 1.0
params.show_field_scale_2 = 1.0
params.show_field_diff_abs = True

## Plot analysis

In [ ]:
class Analysis:
    def __init__(self, ana_name, naming, order_list, error_name_list, error_label_list, filename_prefix="", elem_size=None, marker='o', linestyle='-', plot_gradients=True, label=None, color='b', legend_fond_size = None):
        self.ana_name = ana_name
        self.naming = naming
        self.order_list = order_list
        self.error_name_list = error_name_list
        self.error_label_list = error_label_list
        self.filename_prefix = filename_prefix
        self.elem_size = elem_size
        self.plot_gradients = plot_gradients
        self.color = color
        self.marker = marker
        self.linestyle = linestyle
        self.legend_fond_size = legend_fond_size
        self.label = "; "+label if label else ""
        self.data = self._read_analysis_results()
        self._add_total_error()
        self.data_by_order = self._sort_data_by_order()
        self.gradient_sf = 1

    def _read_analysis_results(self):
        return pd.read_csv(f'{self.ana_name}.csv', header=None, names=self.naming,  index_col=False)
    
    def _add_total_error(self):
        # if columns L2norm H1seminorm fluxErr exist in the data add them together to get the total error
        if 'L2norm' in self.data.columns and 'H1seminorm' in self.data.columns and 'fluxErr' in self.data.columns:
            self.data['totalErr'] = self.data['L2norm'] + self.data['H1seminorm'] + self.data['fluxErr']

    def _sort_data_by_order(self):
        return [self.data.query(f'order == {i}') for i in self.order_list]

    def _plot_error(self, plot_order_func, filename_prefix, xlabel, x):
        for (error_name, error_label) in zip(self.error_name_list, self.error_label_list):
            fig, ax = plt.subplots()
            for i, order in enumerate(self.order_list):
                plot_order_func(ax, i, order, error_name, error_label, xlabel, x)
            self._save_plot(f'{filename_prefix}{error_name}')

    def _plot_order_by_variable(self, ax, i, order, error_name, error_label, xlabel, x, color='b', linestyle='-', marker='o', line_label=""):
        y = self.data_by_order[i][str(error_name)]
        self._plot_and_annotate(ax, x, y, order, error_name, error_label, xlabel, color, linestyle, marker, line_label)

    def _plot_and_annotate(self, ax, x, y, order, error_name, error_label, xlabel, color='b', linestyle='-', marker='o', line_label=""):
        if type(x) != list:
            x = x.reset_index(drop=True)
        y = y.reset_index(drop=True)
        line, = ax.plot(x, y, linestyle+marker, label=f'order = {order}{line_label}', color=color)
        if self.plot_gradients:
            gradient, _ = np.polyfit(np.log(x), np.log(y), 1)
            ax.annotate(f'Gradient = {gradient:.{self.gradient_sf}f}', 
                        xy=(x[len(x)//2], y[len(y)//2]),
                        xycoords='data',
                        xytext=(-1, 0),
                        textcoords='offset points',
                        ha='right',
                        va='bottom',
                        )
        ax.set_xscale('log')
        ax.legend(loc='best')
        ax.set_yscale('log')
        ax.set_ylabel(error_label)
        ax.set_xlabel(xlabel)
        ax.grid(True, ls=':')

    def _save_plot(self, filename):
        filename = f'{self.filename_prefix}{filename}'
        plt.tight_layout()
        plt.savefig(f'{filename}.svg')
        plt.savefig(f'{filename}.png')
        plt.savefig(f'{filename}.pdf')

    def plot_both_analyses_by_elem_size(self, extraAnas=[]):
        if self.elem_size is None:
            raise ValueError("elem_size is not provided for one or both analyses")
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        for (error_name, error_label) in zip(self.error_name_list, self.error_label_list):
            fig, ax = plt.subplots()
            self_label = self.label if extraAnas else ""
            filename_part = "compare_" if extraAnas else ""
            for i, (order_self) in enumerate(zip(self.data_by_order)):
                color = colors[i % len(colors)]
                self._plot_order_by_variable(ax, i, self.order_list[i], error_name, error_label, 'Element size', self.elem_size, color=color, linestyle=self.linestyle, marker=self.marker, line_label=self_label)
                for extra_ana in extraAnas:
                    extra_ana._plot_order_by_variable(ax, i, extra_ana.order_list[i], error_name, error_label, 'Element size', self.elem_size, color=color, linestyle=extra_ana.linestyle, marker=extra_ana.marker, line_label=extra_ana.label)
            
            if self.legend_fond_size:
                ax.legend(fontsize=self.legend_fond_size,loc='best')
            self._save_plot(f'elem_size_{filename_part}{error_name}')

    def plot_both_analyses_by_gaussnum(self, extraAnas=[], extraAnas_singles=[], monte_data_list=[]):
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        for (error_name, error_label) in zip(self.error_name_list, self.error_label_list):
            fig, ax = plt.subplots()
            self_label = self.label if extraAnas else ""
            filename_part = "compare_" if extraAnas else ""
            for i, (order_self) in enumerate(zip(self.data_by_order)):
                color = colors[i % len(colors)]
                self._plot_order_by_variable(ax, i, self.order_list[i], error_name, error_label, 'Gauss number', self.data_by_order[i]["gaussnum"], color=color, linestyle=self.linestyle, marker=self.marker, line_label=self_label)
                for extra_ana in extraAnas:
                    extra_ana._plot_order_by_variable(ax, i, extra_ana.order_list[i], error_name, error_label, 'Gauss number', extra_ana.data_by_order[i]["gaussnum"], color=color, linestyle=extra_ana.linestyle, marker=extra_ana.marker, line_label=extra_ana.label)
            for extra_ana in extraAnas_singles:
                extra_ana._plot_and_annotate(ax, extra_ana.data["gaussnum"], extra_ana.data[error_name], extra_ana.data["order"][0], error_name, error_label, 'Gauss number', color=extra_ana.color, linestyle=extra_ana.linestyle, marker=extra_ana.marker, line_label=extra_ana.label)
            for monte_data in monte_data_list:         
                plt.scatter(monte_data["gaussnum"], monte_data[error_name], marker='x', label='Monte Carlo')
            if self.legend_fond_size:
                ax.legend(fontsize=self.legend_fond_size,loc='best')
            self._save_plot(f'gaussnum_{filename_part}{error_name}')

### Read_med and plot

In [ ]:
def readMed(params):

    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + ".h5m"    
    !{read_med} -med_file {params.med_file} -output_file {h5m_file} -meshsets_config {params.config_file} -dim 2 -adj_dim 1 -log_sl error
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return

## Square_top analysis functions

In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareTop(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_TOP boundary condition
        data = ['[SET_ATTR_FLUX_SQUARE_TOP]', 'number_of_attributes=1', 'user1='+str(params.conductivity), ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_UNIFORM boundary condition set to 0
        data = ['[SET_ATTR_PRESSURE_UNIFORM_0]', 'number_of_attributes=1', 'user1=0.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
# gmsh creation of a 3D beam + visualisation of it
def generateMesh_squareTop(params):
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)

    square1 = gmsh.model.occ.add_rectangle(0, 0, 0, params.length_x, params.length_y)

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    # # ensuring a structured mesh with required element size 
    N = int(params.length_x / params.element_size) + 1

    for n in range(len(gmsh.model.getEntities(1))):
        gmsh.model.mesh.setTransfiniteCurve(n+1, N,'Progression', 1.0)

    gmsh.model.mesh.setTransfiniteSurface(square1)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(1, [3], name="FLUX_SQUARE_TOP")
    gmsh.model.addPhysicalGroup(1, [1,2,4], name="PRESSURE_UNIFORM_0")
    gmsh.model.addPhysicalGroup(2, [square1], name="DOMAIN")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 3D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    med_file = params.mesh_file + ".med"
    gmsh.write(med_file)
    
    # close gmsh
    gmsh.finalize()
    
    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + ".h5m"    
    !{read_med} -med_file {med_file} -output_file {h5m_file} -meshsets_config {params.config_file} -dim 2 -adj_dim 1 -log_sl error
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return


## Square Sin Cos analysis


In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareSinCos(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_FLUX_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1='+str(params.conductivity), ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_PRESSURE_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # SOURCE_SQUARE_SINCOS
        data = ['[SET_ATTR_SOURCE_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
# gmsh creation of a 3D beam + visualisation of it
def generateMesh_squareSinCos(params):
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)

    square1 = gmsh.model.occ.add_rectangle(0, 0, 0, params.length_x, params.length_y)

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    # # ensuring a structured mesh with required element size 
    N = int(params.length_x / params.element_size) + 1

    for n in range(len(gmsh.model.getEntities(1))):
        gmsh.model.mesh.setTransfiniteCurve(n+1, N,'Progression', 1.0)

    gmsh.model.mesh.setTransfiniteSurface(square1)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(1, [3], name="FLUX_SQUARE_SINCOS")
    gmsh.model.addPhysicalGroup(1, [1,2,4], name="PRESSURE_SQUARE_SINCOS")
    gmsh.model.addPhysicalGroup(2, [square1], name="SOURCE_SQUARE_SINCOS")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 3D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    med_file = params.mesh_file + ".med"
    gmsh.write(med_file)
    
    # close gmsh
    gmsh.finalize()
    
    # translate .med file to a format readable by MoFEM and assign values to physical groups
    h5m_file=params.mesh_file + ".h5m"    
    !{read_med} -med_file {med_file} -output_file {h5m_file} -meshsets_config {params.config_file} -dim 2 -adj_dim 1 -log_sl error
    
    # visualise the mesh
    if params.show_mesh:
        vtk_file=params.mesh_file + ".vtk"
        !mbconvert {h5m_file} {vtk_file}

        mesh = pv.read(vtk_file)
        mesh = mesh.shrink(0.98)

        p = pv.Plotter(notebook=True)
        p.add_mesh(mesh, smooth_shading=False)

        p.camera_position = "xy"
        p.show(jupyter_backend='ipygany')
    
    return


## Square NONLINEAR Sin Cos analysis

In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareNonlinearSinCos(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_FLUX_SQUARE_NONLINEAR_SINCOS]', 'number_of_attributes=1', 'user1='+str(params.conductivity), ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_PRESSURE_SQUARE_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # SOURCE_SQUARE_SINCOS
        data = ['[SET_ATTR_SOURCE_SQUARE_NONLINEAR_SINCOS]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
# gmsh creation of a 3D beam + visualisation of it
def generateMesh_squareNonlinearSinCos(params):
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)

    square1 = gmsh.model.occ.add_rectangle(0, 0, 0, params.length_x, params.length_y)

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.occ.synchronize()

    # # ensuring a structured mesh with required element size 
    N = int(params.length_x / params.element_size) + 1

    for n in range(len(gmsh.model.getEntities(1))):
        gmsh.model.mesh.setTransfiniteCurve(n+1, N,'Progression', 1.0)

    gmsh.model.mesh.setTransfiniteSurface(square1)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(1, [3], name="FLUX_SQUARE_NONLINEAR_SINCOS")
    gmsh.model.addPhysicalGroup(1, [1,2,4], name="PRESSURE_SQUARE_SINCOS")
    gmsh.model.addPhysicalGroup(2, [square1], name="SOURCE_SQUARE_NONLINEAR_SINCOS")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 3D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    params.med_file = params.mesh_file + ".med"
    gmsh.write(params.med_file)
    
    # close gmsh
    gmsh.finalize()
    
    readMed(params)
    
    return


## Mexi hat analysis

In [ ]:
# generation of a config file - what attributes should the blocksets have
def generateConfig_squareMexiHat(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_FLUX_UNIFORM]', 'number_of_attributes=1', 'user1=0.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_SQUARE_SINCOS boundary condition
        data = ['[SET_ATTR_PRESSURE_UNIFORM]', 'number_of_attributes=1', 'user1=0.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # SOURCE_SQUARE_SINCOS
        data = ['[SET_ATTR_SOURCE_MEXI]', 'number_of_attributes=1', 'user1=1.0', ' ']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
def generateMesh_squareMexiHat(params):
    
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)
    
    tol = 0.1
    L = params.length_x
    point1 = gmsh.model.geo.addPoint(-L/2, -L/2, 0, tol)
    point2 = gmsh.model.geo.addPoint(L/2, -L/2, 0, tol)
    point3 = gmsh.model.geo.addPoint(L/2, L/2, 0, tol)
    point4 = gmsh.model.geo.addPoint(-L/2, L/2, 0, tol)

    line1 = gmsh.model.geo.addLine(point1, point2)
    line2 = gmsh.model.geo.addLine(point2, point3)
    line3 = gmsh.model.geo.addLine(point3, point4)
    line4 = gmsh.model.geo.addLine(point4, point1)

    curve_loop = gmsh.model.geo.addCurveLoop([line1, line2, line3, line4])
    plane_surface = gmsh.model.geo.addPlaneSurface([curve_loop])
    
    N = int(L / params.element_size) + 1
    
    gmsh.model.geo.mesh.setTransfiniteCurve(line1, N)
    gmsh.model.geo.mesh.setTransfiniteCurve(line2, N)
    gmsh.model.geo.mesh.setTransfiniteCurve(line3, N)
    gmsh.model.geo.mesh.setTransfiniteCurve(line4, N)
    gmsh.model.geo.mesh.setTransfiniteSurface(plane_surface)
    
    gmsh.model.geo.synchronize()

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(2, [plane_surface], name="SOURCE_MEXI")
    # gmsh.model.addPhysicalGroup(1, [1, 2, 3, 4], name="PRESSURE_UNIFORM")
    gmsh.model.addPhysicalGroup(1, [1, 2, 3], name="PRESSURE_UNIFORM")
    gmsh.model.addPhysicalGroup(1, [4], name="FLUX_UNIFORM")

    # for possible square meshes
    if not params.triangle_mesh:
        gmsh.option.setNumber("Mesh.RecombineAll", 1)

    # generate a 2D mesh
    gmsh.model.mesh.generate(2)

    # save as a .med file
    params.med_file = params.mesh_file + ".med"
    gmsh.write(params.med_file)
    
    # close gmsh
    gmsh.finalize()
    
    readMed(params)
    
    return

## L shape analysis

In [ ]:
def generateConfig_Lshape(params):
    # Open the file for writing
    with open(params.config_file, 'w') as f:
        # FLUX_L boundary condition
        data = ['[SET_ATTR_FLUX_L]', 'number_of_attributes=1', 'user1=1']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_UNIFORM boundary condition
        data = ['[SET_ATTR_PRESSURE_UNIFORM]', 'number_of_attributes=1', 'user1=0']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)
        # PRESSURE_L boundary condition
        data = ['[SET_ATTR_PRESSURE_L]', 'number_of_attributes=1', 'user1=1']
        # Use a for loop to write each line of data to the file
        for line in data:
            f.write(line + '\n')
            # print the data as it is written to the file
            print(line)

In [ ]:
def generateMesh_Lshape(params):
    
    # Initialize gmsh
    gmsh.initialize()
    gmsh.option.setNumber("General.Verbosity", 3)
    
    tol = 0.001
    L = 1
    
    point1 = gmsh.model.geo.addPoint(0, 0, 0, tol)
    point2 = gmsh.model.geo.addPoint(0, -1, 0, tol)
    point3 = gmsh.model.geo.addPoint(1, -1, 0, tol)
    point4 = gmsh.model.geo.addPoint(1, 1, 0, tol)
    point5 = gmsh.model.geo.addPoint(-1, 1, 0, tol)
    point6 = gmsh.model.geo.addPoint(-1, 0, 0, tol)
    
    line1 = gmsh.model.geo.addLine(point1, point2);
    line2 = gmsh.model.geo.addLine(point2, point3);
    line3 = gmsh.model.geo.addLine(point3, point4);
    line4 = gmsh.model.geo.addLine(point4, point5);
    line5 = gmsh.model.geo.addLine(point5, point6);
    line6 = gmsh.model.geo.addLine(point6, point1);
    
    curve_loop_1 = gmsh.model.geo.addCurveLoop([line1, line2,line3,line4,line5,line6])
    plane_surface_1 = gmsh.model.geo.addPlaneSurface([curve_loop_1])

    # Create the relevant Gmsh data structures from Gmsh model.
    gmsh.model.geo.synchronize()

    gmsh.option.setNumber("Mesh.MeshSizeMin", params.element_size)
    gmsh.option.setNumber("Mesh.MeshSizeMax", params.element_size)

    # gmsh.model.addPhysicalGroup(dimention, [number of element], name="name")
    gmsh.model.addPhysicalGroup(2, [1], name="DOMAIN")
    gmsh.model.addPhysicalGroup(1, [line1,line6], name="PRESSURE_L")
    gmsh.model.addPhysicalGroup(1, [line2,line3,line4,line5], name="FLUX_L")

    # generate a 2D mesh
    gmsh.model.mesh.generate(2)
    
    # save as a .med file
    params.med_file = params.mesh_file + ".med"
    gmsh.write(params.med_file)
    
    # close gmsh
    gmsh.finalize()
    
    readMed(params)
    
    return